# The Assignment of Week 3
### Import all the libraries I need in this notebook

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for dataframe
import requests # library to handle requests
from bs4 import BeautifulSoup # library for web scrapting

print('Libraries imported.')

Libraries imported.


### web scrapting
   *Get the table from wikipedia  
   *Assign three list to store the data

In [2]:
# assign the website address
web_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(web_url, 'lxml')

# prepare for scrapting
postalcodes, borough, neighborhood = [], [], [] #assign

#start to scrapting
for items in soup.find('table', class_ = 'wikitable sortable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        # appending data to the list
        postalcodes.append(data[0].text)
        borough.append(data[1].text)
        neighborhood.append(data[1].find_next_sibling().text.rstrip('\n'))
    except IndexError:pass

### Transfer Table to Dataframe
   *Drop all the rows have "Not Assigned" value

In [3]:
df_dict = {'PostalCode': postalcodes, 'Borough': borough, 'Neighborhood': neighborhood}
df = pd.DataFrame(data=df_dict)
df.drop(df[(df['Borough'] == 'Not assigned') | (df['Neighborhood'] == 'Not assigned')].index, inplace=True)
df = df.reset_index(drop=True).groupby(['PostalCode','Borough'], sort=False, as_index = False).agg( ','.join)

### Result

In [4]:
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M1B,Scarborough,"Rouge,Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,"Woodbine Gardens,Parkview Hill"
8,M5B,Downtown Toronto,"Ryerson,Garden District"
9,M6B,North York,Glencairn
